In [13]:
import math
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed
from two_weights import *
import matplotlib.pyplot as plt

In [14]:
f = open('Input_Data/product_distributions.txt', 'r')
text = f.read()
f.close()
distributions = text.split('\n')

In [15]:
for i in distributions:
    exec(i.lstrip().rstrip())

In [16]:
distribution_mat = np.array([[p1_mean, p1_sd],
                             [p2_mean, p2_sd],
                             [p3_mean, p3_sd],
                             [p4_mean, p4_sd]])

In [17]:
## 2 MPs, 3 DCs, 4 MZs
## 2 Products, 2 Outsourced Facilities 

num_Scenarios = 21
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2

# Cost of opening
f_i = [2500, 750]
f_j = [1250, 1750, 750]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 15000
Capacities_i[1] = 5000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 7500
Capacities_j[1] = 10000
Capacities_j[2] = 5000
Capacities_l = np.random.randint(500,1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(15, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(20, 25, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(2,3,(Products))

# unit cost of lost sales 
np.random.seed(40)
lost_sales = np.random.randint(18, 25,(Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(0)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [18]:
# save files 
np.savetxt('Input_Data/Instance_1/OpenMP_1.txt', f_i)
np.savetxt('Input_Data/Instance_1/OpenDC_1.txt', f_j)
np.savetxt('Input_Data/Instance_1/Manufacturing_1.txt', Manufacturing_costs)
np.savetxt('Input_Data/Instance_1/TransMPDC_1.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt('Input_Data/Instance_1/TransDCMZ_1.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt('Input_Data/Instance_1/CapacitiesMP_1.txt', Capacities_i)
np.savetxt('Input_Data/Instance_1/CapacitiesDC_1.txt', Capacities_j)
np.savetxt('Input_Data/Instance_1/CapacitiesOutsource_1.txt', Capacities_l)
np.savetxt('Input_Data/Instance_1/SupplierCost_1.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt('Input_Data/Instance_1/TransSupplierDC_1.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt('Input_Data/Instance_1/TransSupplierMZ_1.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt('Input_Data/Instance_1/Volume_1.txt', volume)
np.savetxt('Input_Data/Instance_1/LostSales_1.txt', lost_sales)
np.savetxt('Input_Data/Instance_1/Demand_1.txt', demand.reshape((num_Scenarios*Products, Market)))

In [19]:
## 3 MPs, 4 DCs, 6 MZs
# 2 Products, # 2 Outsourced Facilities

num_Scenarios = 105

Manufacturing_plants = 3
Distribution = 4
Market = 6
Products = 2
Outsourced = 2

# Cost of opening
f_i = [2500, 750, 1750]
f_j = [1250, 1750, 750, 2500]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 15000
Capacities_i[1] = 5000
Capacities_i[2] = 10000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 7500
Capacities_j[1] = 10000
Capacities_j[2] = 5000
Capacities_j[3] = 15000
Capacities_l = np.random.randint(500,1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(15, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(20, 25, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(2,3,(Products))

# unit cost of lost sales 
np.random.seed(40)
lost_sales = np.random.randint(18, 25,(Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(1)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [20]:
np.savetxt('Input_Data/Instance_2/OpenMP_2.txt', f_i)
np.savetxt('Input_Data/Instance_2/OpenDC_2.txt', f_j)
np.savetxt('Input_Data/Instance_2/Manufacturing_2.txt', Manufacturing_costs)
np.savetxt('Input_Data/Instance_2/TransMPDC_2.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt('Input_Data/Instance_2/TransDCMZ_2.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt('Input_Data/Instance_2/CapacitiesMP_2.txt', Capacities_i)
np.savetxt('Input_Data/Instance_2/CapacitiesDC_2.txt', Capacities_j)
np.savetxt('Input_Data/Instance_2/CapacitiesOutsource_2.txt', Capacities_l)
np.savetxt('Input_Data/Instance_2/SupplierCost_2.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt('Input_Data/Instance_2/TransSupplierDC_2.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt('Input_Data/Instance_2/TransSupplierMZ_2.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt('Input_Data/Instance_2/Volume_2.txt', volume)
np.savetxt('Input_Data/Instance_2/LostSales_2.txt', lost_sales)
np.savetxt('Input_Data/Instance_2/Demand_2.txt', demand.reshape((num_Scenarios*Products, Market)))

In [21]:
## 4 MPs, 6 DCs, 8 MZs
# 4 Products, # 4 Outsourced Facilities

num_Scenarios = 200
Manufacturing_plants = 4
Distribution = 6
Market = 8
Products = 4
Outsourced = 4

# Cost of opening
f_i = [2500, 750, 1750, 1250]
f_j = [1250, 1750, 750, 2500, 1250, 750]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 15000
Capacities_i[1] = 5000
Capacities_i[2] = 10000
Capacities_i[3] = 7500
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 7500
Capacities_j[1] = 10000
Capacities_j[2] = 5000
Capacities_j[3] = 15000
Capacities_j[4] = 7500
Capacities_j[5] = 5000
Capacities_l = np.random.randint(500,1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(15, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(20, 25, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(2,3,(Products))

# unit cost of lost sales 
np.random.seed(40)
lost_sales = np.random.randint(18, 25,(Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(2)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [22]:
np.savetxt('Input_Data/Instance_3/OpenMP_3.txt', f_i)
np.savetxt('Input_Data/Instance_3/OpenDC_3.txt', f_j)
np.savetxt('Input_Data/Instance_3/Manufacturing_3.txt', Manufacturing_costs)
np.savetxt('Input_Data/Instance_3/TransMPDC_3.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt('Input_Data/Instance_3/TransDCMZ_3.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt('Input_Data/Instance_3/CapacitiesMP_3.txt', Capacities_i)
np.savetxt('Input_Data/Instance_3/CapacitiesDC_3.txt', Capacities_j)
np.savetxt('Input_Data/Instance_3/CapacitiesOutsource_3.txt', Capacities_l)
np.savetxt('Input_Data/Instance_3/SupplierCost_3.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt('Input_Data/Instance_3/TransSupplierDC_3.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt('Input_Data/Instance_3/TransSupplierMZ_3.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt('Input_Data/Instance_3/Volume_3.txt', volume)
np.savetxt('Input_Data/Instance_3/LostSales_3.txt', lost_sales)
np.savetxt('Input_Data/Instance_3/Demand_3.txt', demand.reshape((num_Scenarios*Products, Market)))

In [23]:
## 6 MPs, 8 DCs, 10 MZs
# 4 Products, # 4 Outsourced Facilities

num_Scenarios = 500
Manufacturing_plants = 6
Distribution = 8
Market = 10
Products = 4
Outsourced = 4

# Cost of opening
f_i = [2500, 750, 1750, 1250, 2500, 750]
f_j = [1250, 1750, 750, 2500, 1250, 750, 2500, 1750]

# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = 15000
Capacities_i[1] = 5000
Capacities_i[2] = 10000
Capacities_i[3] = 7500
Capacities_i[4] = 15000
Capacities_i[5] = 5000
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = 7500
Capacities_j[1] = 10000
Capacities_j[2] = 5000
Capacities_j[3] = 15000
Capacities_j[4] = 7500
Capacities_j[5] = 5000
Capacities_j[6] = 15000
Capacities_j[7] = 10000
Capacities_l = np.random.randint(500,1000, (Products,Outsourced)) # in terms of products 

# Cost of purchasing product m from supplier l 
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(15, 20, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(20, 25, (Products, Outsourced))

# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

# Product volume 
np.random.seed(0)
volume = np.random.uniform(2,3,(Products))

# unit cost of lost sales 
np.random.seed(40)
lost_sales = np.random.randint(18, 25,(Market,Products))

# Demand
demand = np.zeros((num_Scenarios, Products, Market))
np.random.seed(3)
for s in range(num_Scenarios):
    for k in range(Market):
        for m in range(Products):
            demand[s][m][k] = np.round(np.random.normal(distribution_mat[m][0], distribution_mat[m][1]))

In [24]:
np.savetxt('Input_Data/Instance_4/OpenMP_4.txt', f_i)
np.savetxt('Input_Data/Instance_4/OpenDC_4.txt', f_j)
np.savetxt('Input_Data/Instance_4/Manufacturing_4.txt', Manufacturing_costs)
np.savetxt('Input_Data/Instance_4/TransMPDC_4.txt', Transportation_i_j.reshape((Products*Manufacturing_plants, Distribution)))
np.savetxt('Input_Data/Instance_4/TransDCMZ_4.txt', Transportation_j_k.reshape((Products*Distribution, Market)))
np.savetxt('Input_Data/Instance_4/CapacitiesMP_4.txt', Capacities_i)
np.savetxt('Input_Data/Instance_4/CapacitiesDC_4.txt', Capacities_j)
np.savetxt('Input_Data/Instance_4/CapacitiesOutsource_4.txt', Capacities_l)
np.savetxt('Input_Data/Instance_4/SupplierCost_4.txt', Supplier_cost.reshape((levels*Products, Outsourced)))
np.savetxt('Input_Data/Instance_4/TransSupplierDC_4.txt', T_O_DC.reshape((Products*Outsourced, Distribution)))
np.savetxt('Input_Data/Instance_4/TransSupplierMZ_4.txt', T_O_MZ.reshape((Products*Outsourced, Market)))
np.savetxt('Input_Data/Instance_4/Volume_4.txt', volume)
np.savetxt('Input_Data/Instance_4/LostSales_4.txt', lost_sales)
np.savetxt('Input_Data/Instance_4/Demand_4.txt', demand.reshape((num_Scenarios*Products, Market)))